In [5]:
# Basic Dependencies
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
def load_year_data(year):
    """
    Load the Best Ball data into a DataFrame, handles 2021 and 2022 differences in file structure
    """
    df = []
    # Determine the subdirectory based on the year
    subdir = '' if year == 2021 else '/fast'

    # Load regular season data
    for file in os.listdir(f'../data/{year}/regular_season{subdir}'):
        df.append(pd.read_csv(f'../data/{year}/regular_season{subdir}/{file}'))
    df = pd.concat(df, ignore_index=True)

    # Determine how to load post season data based on the year
    post_season_loaders = {'qf': [], 'sf': None, 'f': None}
    if year == 2021:
        post_season_loaders['qf'].append(pd.read_csv(
            f'../data/{year}/post_season/quarterfinals.csv'))
        post_season_loaders['sf'] = pd.read_csv(
            f'../data/{year}/post_season/semifinals.csv')
        post_season_loaders['f'] = pd.read_csv(
            f'../data/{year}/post_season/finals.csv')
    else:
        for file in os.listdir(f'../data/{year}/post_season/quarterfinals/'):
            post_season_loaders['qf'].append(pd.read_csv(
                f'../data/{year}/post_season/quarterfinals/{file}'))
        post_season_loaders['sf'] = pd.read_csv(
            f'../data/{year}/post_season/semifinals/part_00.csv')
        post_season_loaders['f'] = pd.read_csv(
            f'../data/{year}/post_season/finals/part_00.csv')

    post_season_loaders['qf'] = pd.concat(
        post_season_loaders['qf'], ignore_index=True)

    # Convert playoff dataframes to dictionaries and map onto df
    for k, playoffs_df in post_season_loaders.items():
        playoffs_dict = dict(
            zip(playoffs_df['tournament_entry_id'], [1]*len(playoffs_df)))
        df[k] = df['tournament_entry_id'].map(
            playoffs_dict).fillna(0).astype(int)

    # Add a column for the year
    df['year'] = year

    return df

In [7]:
df21 = load_year_data(2021)
df22 = load_year_data(2022)

In [8]:
# Combine the dataframes
df = pd.concat([df21, df22], ignore_index=True)

In [9]:
# Drop columns that are not needed
df = df.drop(columns=['draft_entry_id', 'tournament_round_draft_entry_id'])

In [10]:
df.head()

,draft_id,draft_time,clock,tournament_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,overall_pick_number,team_pick_number,pick_points,roster_points,playoff_team,qf,sf,f,year
0,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20 05:54:15.422561+00:00,30,e762e1de-c639-431b-bbb6-1e30ee9e291f,1,Cam Akers,RB,11,12.14,10,15,2,0.00,1675.10,0,0,0,0,2021
1,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20 05:54:15.422561+00:00,30,5e1f4e64-41fa-47ab-8c4b-5562fd1e9eb0,1,Brandin Cooks,WR,10,91.48,5,92,8,103.60,1554.92,0,0,0,0,2021
2,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20 05:54:15.422561+00:00,30,f33b8689-4157-409d-950a-0b12244954e2,1,Noah Fant,TE,11,100.64,6,102,9,72.30,1656.58,0,0,0,0,2021
3,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20 05:54:15.422561+00:00,30,aeba7ded-dfeb-4b96-bc59-501c4ca29202,1,Matt Ryan,QB,6,138.23,8,152,13,97.56,1732.12,1,1,0,0,2021
4,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20 05:54:15.422561+00:00,30,072a49a9-bc24-4176-b09b-e053fc9f05eb,1,A.J. Brown,WR,13,23.25,2,23,2,103.50,1614.82,0,0,0,0,2021


In [13]:
df['draft_time'] = pd.to_datetime(df['draft_time'], utc=True)

idx = df[df.groupby(['player_name'])['draft_time'].transform('min') == df['draft_time']].index

player_rankings = df.loc[idx, ['player_name','position_name', 'year', 'projection_adp']].reset_index(drop=True)

In [14]:
player_rankings.head()

,player_name,position_name,year,projection_adp
0,Noah Brown,WR,2021,0.00
1,Adam Shaheen,TE,2021,0.00
2,Kaden Smith,TE,2021,0.00
3,Tim Tebow,TE,2021,215.86
4,Tyler Kroft,TE,2021,0.00


In [89]:
class DraftSimulator:
    def __init__(self, available_players):
        available_players = available_players[available_players['projection_adp'] != 0]
        self.available_players = available_players
        self.num_teams = 12
        self.teams = self._init_teams()

    def _init_teams(self):
        return [
            {
                'QB': None,
                'RB': [None, None],
                'WR': [None, None, None],
                'TE': None,
                'FLEX': None,
                'BENCH': [None]*10
            } for _ in range(self.num_teams)
        ]

    def _best_available_player(self):
        return self.available_players.nsmallest(1, 'projection_adp').iloc[0]

    def _team_dict_to_df(self, team_dict, team_number):
        df = pd.DataFrame()
        for position, player in team_dict.items():
            if isinstance(player, list):
                for idx, p in enumerate(player):
                    if p is None:
                        continue
                    new_row = p.copy()
                    new_row['position'] = f'{position}{idx + 1}'
                    new_row['team_number'] = team_number
                    df = df.append(new_row, ignore_index=True)
            else:
                if player is None:
                    continue
                new_row = player.copy()
                new_row['position'] = position
                new_row['team_number'] = team_number
                df = df.append(new_row, ignore_index=True)
        return df

    def _is_team_full(self, team):
        return all(player is not None for position, players in team.items() for player in (players if isinstance(players, list) else [players]))

    def _get_unfilled_position(self, team, player_position):
        for position in ['QB', 'RB', 'WR', 'TE']:
            if position == player_position:
                if position in ['RB', 'WR'] and any(p is None for p in team[position]):
                    return position
                elif team[position] is None:
                    return position

        if player_position in ['RB', 'WR', 'TE'] and team['FLEX'] is None:
            return 'FLEX'
        
        return 'BENCH'

    def simulate_draft(self):
        pick_number = 0  # Initialize the pick number
        for round in range(17):
            # For each team
            for team_number in range(self.num_teams) if round % 2 == 0 else reversed(range(self.num_teams)):
                team = self.teams[team_number]

                player = self._best_available_player()
                position = self._get_unfilled_position(
                    team, player['position_name'])
                if position is None:
                    continue  # Skip to the next iteration if there are no open slots
                player['team_number'] = team_number
                player['round_number'] = round + 1
                # Assign the current pick number to the player
                player['pick_number'] = pick_number + 1
                pick_number += 1  # Increment the pick number

                if position in ['RB', 'WR', 'BENCH']:
                    empty_position_index = next(
                        i for i, p in enumerate(team[position]) if p is None)
                    team[position][empty_position_index] = player
                else:
                    team[position] = player

                # Remove the chosen player from available players
                self.available_players = self.available_players[
                    self.available_players['player_name'] != player['player_name']]

        teams_df = pd.concat([self._team_dict_to_df(team, team_number+1)
                              for team_number, team in enumerate(self.teams)], ignore_index=True)

        return teams_df

In [92]:
draft = DraftSimulator(available_players=player_rankings)

In [93]:
results = draft.simulate_draft()

results.sort_values('pick_number', ascending=True).head(24)

,player_name,position_name,year,projection_adp,team_number,round_number,pick_number,position
1,Christian McCaffrey,RB,2021.0,1.0,1.0,1.0,1.0,RB1
18,Dalvin Cook,RB,2021.0,2.0,2.0,1.0,2.0,RB1
35,Saquon Barkley,RB,2021.0,3.0,3.0,1.0,3.0,RB1
52,Derrick Henry,RB,2021.0,4.0,4.0,1.0,4.0,RB1
69,Alvin Kamara,RB,2021.0,5.0,5.0,1.0,5.0,RB1
86,Jonathan Taylor,RB,2021.0,6.0,6.0,1.0,6.0,RB1
103,Aaron Jones,RB,2021.0,7.0,7.0,1.0,7.0,RB1
125,Travis Kelce,TE,2021.0,8.0,8.0,1.0,8.0,TE
139,Davante Adams,WR,2021.0,9.0,9.0,1.0,9.0,WR1
154,Ezekiel Elliott,RB,2021.0,10.0,10.0,1.0,10.0,RB1


In [95]:
results[results['team_number']== 3]

,player_name,position_name,year,projection_adp,team_number,round_number,pick_number,position
34,Russell Wilson,QB,2021.0,69.0,3.0,6.0,70.0,QB
35,Saquon Barkley,RB,2021.0,3.0,3.0,1.0,3.0,RB1
36,Zack Moss,RB,2021.0,94.0,3.0,9.0,99.0,RB2
37,DK Metcalf,WR,2021.0,27.0,3.0,3.0,27.0,WR1
38,Chris Godwin,WR,2021.0,44.0,3.0,4.0,46.0,WR2
39,Tee Higgins,WR,2021.0,49.0,3.0,5.0,51.0,WR3
40,George Kittle,TE,2021.0,22.0,3.0,2.0,22.0,TE
41,D.J. Chark,WR,2021.0,73.0,3.0,7.0,75.0,FLEX
42,Brandin Cooks,WR,2021.0,90.0,3.0,8.0,94.0,BENCH1
43,Gus Edwards,RB,2021.0,109.0,3.0,10.0,118.0,BENCH2


In [ ]:
draft_id = '64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9'

draft = Draft(df, draft_id)

test = draft.get_draft()

test.head()

In [ ]:
# Define the positions
positions = ['QB', 'RB', 'TE', 'WR']

# Create a column for each position adp trend
for pos in positions:
    df['adp_trend_' +
        pos] = np.where(df['position_name'] == pos, df['adp_trend'], 0)

In [ ]:
# Get a look at the new adp trend columns
df.head()

,draft_id,draft_time,clock,tournament_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,...,qf,sf,f,year,earliest_adp,adp_trend,adp_trend_QB,adp_trend_RB,adp_trend_TE,adp_trend_WR
0,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,e762e1de-c639-431b-bbb6-1e30ee9e291f,1,Cam Akers,RB,11,12.14,10,...,0,0,0,2021,0.00,20.86,0.00,20.86,0.00,0.00
1,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,5e1f4e64-41fa-47ab-8c4b-5562fd1e9eb0,1,Brandin Cooks,WR,10,91.48,5,...,0,0,0,2021,87.00,-40.75,0.00,0.00,0.00,-40.75
2,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,f33b8689-4157-409d-950a-0b12244954e2,1,Noah Fant,TE,11,100.64,6,...,0,0,0,2021,90.08,17.36,0.00,0.00,17.36,0.00
3,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,aeba7ded-dfeb-4b96-bc59-501c4ca29202,1,Matt Ryan,QB,6,138.23,8,...,1,0,0,2021,101.90,15.07,15.07,0.00,0.00,0.00
4,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,072a49a9-bc24-4176-b09b-e053fc9f05eb,1,A.J. Brown,WR,13,23.25,2,...,0,0,0,2021,15.95,-7.25,0.00,0.00,0.00,-7.25


In [ ]:
# Create a column for delta from ADP
df['market_delta'] = df['projection_adp'] - df['overall_pick_number']

In [ ]:
# Create a column for delta from ADP by position
for pos in positions:
    df['market_delta_' +
        pos] = np.where(df['position_name'] == pos, df['market_delta'], 0)

In [ ]:
df.head()

,draft_id,draft_time,clock,tournament_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,...,adp_trend,adp_trend_QB,adp_trend_RB,adp_trend_TE,adp_trend_WR,market_delta,market_delta_QB,market_delta_RB,market_delta_TE,market_delta_WR
0,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,e762e1de-c639-431b-bbb6-1e30ee9e291f,1,Cam Akers,RB,11,12.14,10,...,20.86,0.00,20.86,0.00,0.00,-2.86,0.00,-2.86,0.00,0.00
1,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,5e1f4e64-41fa-47ab-8c4b-5562fd1e9eb0,1,Brandin Cooks,WR,10,91.48,5,...,-40.75,0.00,0.00,0.00,-40.75,-0.52,0.00,0.00,0.00,-0.52
2,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,f33b8689-4157-409d-950a-0b12244954e2,1,Noah Fant,TE,11,100.64,6,...,17.36,0.00,0.00,17.36,0.00,-1.36,0.00,0.00,-1.36,0.00
3,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,aeba7ded-dfeb-4b96-bc59-501c4ca29202,1,Matt Ryan,QB,6,138.23,8,...,15.07,15.07,0.00,0.00,0.00,-13.77,-13.77,0.00,0.00,0.00
4,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,072a49a9-bc24-4176-b09b-e053fc9f05eb,1,A.J. Brown,WR,13,23.25,2,...,-7.25,0.00,0.00,0.00,-7.25,0.25,0.00,0.00,0.00,0.25


In [ ]:
# note: using adp trend and market delta to understand how the draft is going at x1 point in the draft and use that to 
# guess if the player will be available at point x2 in the draft



,draft_id,draft_time,clock,tournament_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,...,adp_trend,adp_trend_QB,adp_trend_RB,adp_trend_TE,adp_trend_WR,market_delta,market_delta_QB,market_delta_RB,market_delta_TE,market_delta_WR
0,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,e762e1de-c639-431b-bbb6-1e30ee9e291f,1,Cam Akers,RB,11,12.14,10,...,20.86,0.00,20.86,0.00,0.00,-2.86,0.00,-2.86,0.00,0.00
1,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,5e1f4e64-41fa-47ab-8c4b-5562fd1e9eb0,1,Brandin Cooks,WR,10,91.48,5,...,-40.75,0.00,0.00,0.00,-40.75,-0.52,0.00,0.00,0.00,-0.52
2,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,f33b8689-4157-409d-950a-0b12244954e2,1,Noah Fant,TE,11,100.64,6,...,17.36,0.00,0.00,17.36,0.00,-1.36,0.00,0.00,-1.36,0.00
3,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,aeba7ded-dfeb-4b96-bc59-501c4ca29202,1,Matt Ryan,QB,6,138.23,8,...,15.07,15.07,0.00,0.00,0.00,-13.77,-13.77,0.00,0.00,0.00
4,64c5c57f-dfe6-49b7-a0c2-180a8e8e2ad9,2021-07-20,30,072a49a9-bc24-4176-b09b-e053fc9f05eb,1,A.J. Brown,WR,13,23.25,2,...,-7.25,0.00,0.00,0.00,-7.25,0.25,0.00,0.00,0.00,0.25


In [ ]:
# CREATE A SIMPLE WEB APP WHERE USERS CAN PLUG IN EACH PICK AND GET A FEEL FOR IF 
# PEOPLE ARE BUYING LOW / HIGH ON A POSITION OR IF THEY ARE REACHING / SLIPPING ON A POSITION